In [3]:
try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    # !pip install -q -U tfx==0.15.0rc0
    print("You can safely ignore the package incompatibility errors.")
except Exception:
    pass


import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"
print(tf.__version__)

from tensorflow.keras.utils import plot_model
from tensorflow.keras import backend as K
import numpy as np
import matplotlib.pyplot as plt


You can safely ignore the package incompatibility errors.
2.1.0


# This documments is for creating data loader with tf data

## Preprocesse with numpy 
## convert to tf data

In [0]:
from tensorflow.keras.datasets import mnist

In [6]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [0]:
x_train = (x_train - 127.5) / 127.5
x_test = (x_test - 127.5) / 127.5

In [12]:
y_train

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

In [13]:
x_train.shape

(60000, 28, 28)

In [0]:
class preprocess_hyperparmas:
  seed = 1228
  batch_size=256


hp = preprocess_hyperparmas()

In [0]:
train_dataset = tf.data.Dataset.from_tensor_slices ((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=60000, seed=hp.seed)
train_dataset = train_dataset.batch(batch_size=hp.batch_size)

test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_dataset = test_dataset.batch(batch_size=hp.batch_size)

In [21]:
train_dataset

<BatchDataset shapes: ((None, 28, 28), (None,)), types: (tf.float64, tf.uint8)>

In [20]:
counter = 0
for i in train_dataset:
  print(i)
  counter += 1
  if counter == 1:
    break

(<tf.Tensor: shape=(256, 28, 28), dtype=float64, numpy=
array([[[-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.],
        ...,
        [-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.]],

       [[-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.],
        ...,
        [-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.]],

       [[-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.],
        ...,
        [-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.]],

       ...,

       [[-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1

## Create model 
### 1) sequence dense
### 2) class dense 

In [0]:
model_base = tf.keras.models.Sequential()
model_base.add(tf.keras.layers.Flatten(input_shape = (28,28)))
model_base.add(tf.keras.layers.Dense(128, activation='relu'))
model_base.add(tf.keras.layers.Dense(10, activation='softmax'))

In [25]:
model_base.trainable_variables

[<tf.Variable 'dense/kernel:0' shape=(784, 128) dtype=float32, numpy=
 array([[-0.01030801,  0.06420987,  0.03593113, ...,  0.07213207,
          0.00216206, -0.04388396],
        [-0.00645377, -0.01835321,  0.07894803, ...,  0.03019656,
          0.03813768, -0.00012398],
        [-0.06139038, -0.0699673 ,  0.06770866, ...,  0.06191877,
          0.00465985,  0.05382685],
        ...,
        [-0.00371972,  0.02149504, -0.04355917, ..., -0.05915331,
         -0.01168314,  0.00394463],
        [-0.00226243, -0.03350121,  0.02337952, ..., -0.01591289,
         -0.04399214,  0.04678196],
        [-0.031551  , -0.07145801, -0.05147343, ...,  0.07907752,
         -0.00728878,  0.01628308]], dtype=float32)>,
 <tf.Variable 'dense/bias:0' shape=(128,) dtype=float32, numpy=
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [0]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Define metrics

train_loss = tf.keras.metrics.Mean(name='train')
train_acc = tf.keras.metrics.SparseCategoricalAccuracy(name='train_acc')

test_loss = tf.keras.metrics.Mean(name='test')
test_acc = tf.keras.metrics.SparseCategoricalAccuracy(name='test_acc')

In [0]:
@tf.function
def train_step(model, data, labels):
  with tf.GradientTape() as tape:
    y_hat = model(data, training=True)
    loss = loss_object(labels, y_hat)

  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_acc(labels, y_hat)

In [0]:
@tf.function
def test_step(model, data, labels):
  with tf.GradientTape() as tape:
    y_hat = model(data, training=False)
    loss = loss_object(labels, y_hat)

  test_loss(loss)
  test_acc(labels, y_hat)

In [38]:
EPOCHS = 5
for epoch in range(EPOCHS):
  train_loss.reset_states()
  train_acc.reset_states()
  test_loss.reset_states()
  test_acc.reset_states()

  for image, labels in train_dataset:
    train_step(model_base, image, labels)
  
  for t_image, t_labels in test_dataset:
    test_step(model_base, t_image, t_labels)
  
  templete = 'Epoch  {}, loss {}, acc {}, test loss {}, test acc {}'
  print(templete.format(epoch+1,
                        train_loss.result(),
                        train_acc.result()*100,
                        test_loss.result(),
                        test_acc.result()*100))

Epoch  1, loss 0.1879865825176239, acc 94.49666595458984, test loss 0.16146795451641083, test acc 95.36000061035156
Epoch  2, loss 0.15363655984401703, acc 95.47833251953125, test loss 0.15807147324085236, test acc 95.3499984741211
Epoch  3, loss 0.12913818657398224, acc 96.21499633789062, test loss 0.1312759965658188, test acc 96.1300048828125
Epoch  4, loss 0.11208085715770721, acc 96.68833923339844, test loss 0.10663650184869766, test acc 96.73999786376953
Epoch  5, loss 0.09948568046092987, acc 97.03500366210938, test loss 0.10157375037670135, test acc 96.79000091552734


In [41]:
model_base = tf.keras.models.Sequential()
model_base.add(tf.keras.layers.Flatten(input_shape = (28,28)))
model_base.add(tf.keras.layers.Dense(128, activation='relu'))
model_base.add(tf.keras.layers.Dense(10, activation='softmax'))

model_base.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['acc'])

model_base.fit(train_dataset, epochs=5, validation_data=test_dataset)

Train for 235 steps, validate for 40 steps
Epoch 1/5
235/235 [==============================] - 2s 7ms/step - loss: 0.4500 - acc: 0.8673 - val_loss: 0.2512 - val_acc: 0.9280
Epoch 2/5
235/235 [==============================] - 1s 6ms/step - loss: 0.2328 - acc: 0.9325 - val_loss: 0.1951 - val_acc: 0.9415
Epoch 3/5
235/235 [==============================] - 1s 6ms/step - loss: 0.1781 - acc: 0.9476 - val_loss: 0.1662 - val_acc: 0.9507
Epoch 4/5
235/235 [==============================] - 1s 6ms/step - loss: 0.1450 - acc: 0.9574 - val_loss: 0.1301 - val_acc: 0.9605
Epoch 5/5
235/235 [==============================] - 1s 6ms/step - loss: 0.1198 - acc: 0.9649 - val_loss: 0.1139 - val_acc: 0.9658


## Create Custom layer

In [0]:
class mylayer(tf.keras.layers.Layer):
  def __init__(self, units=32, input_dim=32, **kwargs):
    super(mylayer, self).__init__(*kwargs)
    w_init = tf.random_normal_initializer()
    self.w = tf.Variable(initial_value=w_init(shape = (input_dim, units), dtype='float32'), trainable=True)

    b_init = tf.zeros_initializer()
    self.b = tf.Variable(initial_value=b_init(shape = (units, ), dtype='float32'), trainable=True)

  def call(self, inputs):
    return tf.matmul(inputs, self.w) + self.b
  
  def call(self, inputs):
    return tf.matmul(inputs, self.w) + self.b


In [0]:
linear = mylayer(4,2)

In [141]:
linear.trainable_variables

[<tf.Variable 'Variable:0' shape=(2, 4) dtype=float32, numpy=
 array([[ 0.01259459, -0.0282155 , -0.04189282,  0.04551737],
        [ 0.04347293, -0.01694804, -0.00612204, -0.01912688]],
       dtype=float32)>,
 <tf.Variable 'Variable:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>]

In [0]:
x = tf.ones((2,2))

In [45]:
x

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[1., 1.],
       [1., 1.]], dtype=float32)>

In [47]:
linear = mylayer(4, 2)


In [0]:
y = linear(x)

In [49]:
y

<tf.Tensor: shape=(2, 4), dtype=float32, numpy=
array([[-7.9482794e-05, -7.5685799e-02, -2.5440868e-02, -6.5402210e-02],
       [-7.9482794e-05, -7.5685799e-02, -2.5440868e-02, -6.5402210e-02]],
      dtype=float32)>

## Creating Custom layer

In [0]:
layer = tf.keras.layers.Dense(100)

In [0]:
class MyDenseLayer(tf.keras.layers.Layer):
  def __init__ (self, num_outputs, **kwargs):
    super(MyDenseLayer, self).__init__(**kwargs)
    self.num_outputs = num_outputs

  def build(self, input_shape):
    self.kernel = self.add_weight('kernel', 
                                    shape = [int(input_shape[-1]), self.num_outputs])
    
  def call(self, input):
    return tf.matmul(input, self.kernel)



In [0]:
layer = MyDenseLayer(10)

In [0]:
_ = layer(tf.zeros([10,5]))

In [166]:
[var.name for var in layer.trainable_variables]

['my_dense_layer_2/kernel:0']

In [0]:
class ResnetIdentityBlock(tf.keras.Model):
  def __init__(self, kernel_size, filters):
    super(ResnetIdentityBlock, self).__init__(name='')
    filters1, filters2, filters3 = filters

    self.conv2a = tf.keras.layers.Conv2D(filters1, 1)
    self.bn2a = tf.keras.layers.BatchNormalization()

    self.conv2b = tf.keras.layers.Conv2D(filters2, kernel_size, padding='same')
    self.bn2b = tf.keras.layers.BatchNormalization()

    self.conv2c = tf.keras.layers.Conv2D(filters3, 1)
    self.bn2c = tf.keras.layers.BatchNormalization()

  def call(self, input_tensor, training=False):
    x = self.conv2a(input_tensor)
    x = self.bn2a(x, training=training)
    x = tf.nn.relu(x)

    x = self.conv2b(x)
    x = self.bn2b(x, training=training)
    x = tf.nn.relu(x)
    
    x = self.conv2c(x)
    x = self.bn2c(x, training=training)

    x = x + input_tensor
    return tf.nn.relu(x)

In [0]:
block = ResnetIdentityBlock(1, [1,2,3])

In [0]:
_ = block(tf.zeros([1, 2, 3, 3])) 

In [180]:
block.layers

In [181]:
len(block.variables)

18

In [182]:
block.summary()

Model: "resnet_identity_block_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            multiple                  4         
_________________________________________________________________
batch_normalization_9 (Batch multiple                  4         
_________________________________________________________________
conv2d_10 (Conv2D)           multiple                  4         
_________________________________________________________________
batch_normalization_10 (Batc multiple                  8         
_________________________________________________________________
conv2d_11 (Conv2D)           multiple                  9         
_________________________________________________________________
batch_normalization_11 (Batc multiple                  12        
Total params: 41
Trainable params: 29
Non-trainable params: 12
______________________________________________

In [0]:
class CustomDense(tf.keras.layers.Layer):
  def __init__(self, units=32):
    super(CustomDense, self).__init__()
    self.units = units
  
  def build(self, input_shape):
    self.w = self.add_weight(shape = (input_shape[-1], self.units),
                             initializer='random_normal',
                             trainable=True)
    self.b = self.add_weight(shape = (self.units, ),
                             initializer='zeros',
                             trainable=True)
    
  def call(self, inputs):
    return tf.matmul(inputs, self.w) + self.b

## Create Custom Attention

In [0]:
class Attention(tf.keras.layers.Layer):
  def __init__(self, step_dim, W_regularizer=None, b_regularizer=None, W_constraint=None, b_constraint=None, bias=True, **kwargs):
    super(Attention, self).__init__(**kwargs)
    self.supports_masking = True
    self.init = tf.keras.initializers.get('glorot_uniform')
    self.W_regularizer = tf.keras.regularizers.get(W_regularizer)
    self.b_regularizer = tf.keras.regularizers.get(b_regularizer)

    self.W_constraint = tf.keras.constraints.get(W_constraint)
    self.b_constraint = tf.keras.constraints.get(b_constraint)

    self.bias = bias
    self.step_dim = step_dim
    self.feature_dim = 0

  def build(self, input_shape):
    assert len(input_shape) == 3
    self.feature_dim = input_shape[-1]

    self.W = self.add_weight(shape = (input_shape[-1], ),
                             initializer = self.init,
                             regularizer = self.W_regularizer,
                             constraint = self.W_constraint,
                             name = '{}_w'.format(self.name))
    if self.bias:
      self.b = self.add_weight(shape = (input_shape[-1], ),
                               initializer='zero', 
                               regularizer = self.b_regularizer,
                               constraint = self.b_constraint,
                               name = '{}_b'.format(self.name))
    else:
      self.b = None


  def compute_mask(self, input, input_mask=None):
    return None

  def call(self, x, mask=None):
    features_dim = self.feature_dim
    step_dim = self.step_dim

    eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)), 
                           K.reshape(self.W, (features_dim, 1))), (-1, step_dim))
    
    if self.bias:
      eij += self.b

    eij = K.tanh(eij)

    a = K.exp(eij)

    if mask is not None:
      a = a * K.cast(mask, K.floatx())
    
    a = a / K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

    a = K.expand_dims(a)
    weighted_input = x * a
    return K.sum(weighted_input, axis=1)
  
  def compute_output_shape(self, input_shape):
    return input_shape[0], self.features_dim

    